### Import libraries, mount drive, etc.

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My\ Drive/Colab\ Notebooks/NeuralODEs_ROM_Closure/neuralDDE_ROM_Closure

In [ ]:
%pip install quadpy

In [ ]:
from src.utilities.DDE_Solver import ddeinttf
import src.solvers.neuralDistDDE_with_adjoint as nddde

import quadpy

import time
import sys
import os
from IPython.core.debugger import set_trace

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy import integrate
import scipy.interpolate
from shutil import move

tf.keras.backend.set_floatx('float32')
import logging
tf.get_logger().setLevel(logging.ERROR)

## Define some useful classes

### Define a custom loss function

In [ ]:
class custom_loss(tf.keras.losses.Loss):

    def call(self, true_y, pred_y):
        loss = tf.reduce_mean(tf.sqrt(tf.reduce_sum(tf.math.squared_difference(pred_y, true_y), axis=-1)), axis=0)
        return loss

### Define a custom plotting function

In [ ]:
class custom_plot:

    def __init__(self, true_y, t, figsave_dir, args):
        self.true_y = true_y
        self.t = t
        self.figsave_dir = figsave_dir
        self.args = args

    def plot(self, *pred_y, epoch = 0):
        fig = plt.figure(figsize=(12, 4), facecolor='white')
        ax_traj = fig.add_subplot(121, frameon=False)
        ax_phase = fig.add_subplot(122, frameon=False)

        ax_traj.cla()
        ax_traj.set_title('Trajectories')
        ax_traj.set_xlabel('t')
        ax_traj.set_ylabel('x,y')
        ax_traj.plot(self.t.numpy(), self.true_y.numpy()[:, 0, 0], 'b-', self.t.numpy(), self.true_y.numpy()[:, 0, 1], 'g-')
        ax_traj.set_xlim(min(t.numpy()), max(t.numpy()))
        ax_traj.set_ylim(-1, 1)

        ax_phase.cla()
        ax_phase.set_title('Phase Portrait')
        ax_phase.set_xlabel('x')
        ax_phase.set_ylabel('y')
        ax_phase.plot(self.true_y.numpy()[:, 0, 0], self.true_y.numpy()[:, 0, 1], 'g-')
        ax_phase.set_xlim(-1, 1)
        ax_phase.set_ylim(-1, 1)  

        if epoch != 0 or self.args.restart == 1 :
            ax_traj.plot(self.t.numpy(), pred_y[0].numpy()[:, 0, 0], 'b--', self.t.numpy(), pred_y[0].numpy()[:, 0, 1], 'g--',)
            ax_phase.plot(pred_y[0].numpy()[:, 0, 0], pred_y[0].numpy()[:, 0, 1], 'g--')

        plt.show() 

        if epoch != 0: 
            fig.savefig(os.path.join(self.figsave_dir, 'img'+str(epoch)))

### Define neural net architectures

#### Main network

In [ ]:
class DDEFuncMain(tf.keras.Model):

    def __init__(self, **kwargs):
        super(DDEFuncMain, self).__init__(**kwargs)
        
        self.out = tf.keras.layers.Dense(args_nddde.state_dim, activation='linear',
                                       kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.1), use_bias=True)

    def call(self, z):
        for i in range(len(self.layers)):
            z = self.layers[i](z)
        return z

#### Auxilary network

In [ ]:
class DDEFuncAux(tf.keras.Model):

    def __init__(self, **kwargs):
        super(DDEFuncAux, self).__init__(**kwargs)
        
        self.out = tf.keras.layers.Dense(args_nddde.state_dim, activation='linear',
                                       kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.1), use_bias=True)

    def call(self, z):
        for i in range(len(self.layers)):
            z = self.layers[i](z)
        return z

### Initialize model parameters

In [ ]:
args_nddde = nddde.nddde_arguments(data_size = 1000, batch_time = 12, batch_time_skip = 2, batch_size = 5, epochs = 500, learning_rate = 0.075, decay_rate = 0.95, test_freq = 1, plot_freq = 2, 
                 d_max = 1., nn_d1 = 0., nn_d2 = 0.5, state_dim = 2, adj_data_size = 2,
                 model_dir = 'DistDDE_runs/model_dir_test', restart = 1, val_percentage = 0.2)

### Make a copy of the current script

In [ ]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/NeuralODEs_ROM_Closure

if not os.path.exists(args_nddde.model_dir):
  os.makedirs(args_nddde.model_dir)

checkpoint_dir_main = os.path.join(args_nddde.model_dir, "ckpt_main")
checkpoint_dir_aux = os.path.join(args_nddde.model_dir, "ckpt_aux")
checkpoint_prefix_main = os.path.join(checkpoint_dir_main, "ckpt")
checkpoint_prefix_aux = os.path.join(checkpoint_dir_aux, "ckpt")
if not os.path.exists(checkpoint_dir_main):
  os.makedirs(checkpoint_dir_main)
if not os.path.exists(checkpoint_dir_aux):
  os.makedirs(checkpoint_dir_aux)

figsave_dir = os.path.join(args_nddde.model_dir, "img")
if not os.path.exists(figsave_dir):
  os.makedirs(figsave_dir)

!jupyter nbconvert --to script 'neuralDDE_ROM_Closure/examples/neuralDistDDE_Example.ipynb'
move('neuralDDE_ROM_Closure/examples/neuralDistDDE_Example.txt', os.path.join(args_nddde.model_dir, "orig_run_file.py"))

### Define initial conditions and other parameters associated with the true DistDDE

In [ ]:
class initial_cond(tf.keras.Model):

    def call(self, t):
        return tf.convert_to_tensor([[1., 0.]], dtype=tf.float32)

class true_eqn_integrate:
    def __init__(self, B, y, t_lowerlim, t_upperlim):
        self.B = B
        self.y = y
        self.t_ll = t_lowerlim
        self.t_ul = t_upperlim
        self.scheme = quadpy.c1.gauss_legendre(5)
        self.integ = self.integrate_By()

    def By(self, t):

        return tf.einsum('ab, cb -> ca', tf.cast(tf.transpose(self.B), tf.float64), tf.cast(self.y(t), tf.float64)).numpy()

    def stack_By(self, t):
        return np.stack([self.By(t[i]) for i in range(len(t))], axis=-1)

    def integrate_By(self):
        return tf.convert_to_tensor(self.scheme.integrate(self.stack_By, [self.t_ll, self.t_ul]), tf.float64)

true_z0 = initial_cond() # Initial conditions
t = tf.linspace(0., 10., args_nddde.data_size) # Time array
true_A = tf.convert_to_tensor([[-0.1, 2.0], [-2.0, -0.1]], dtype=tf.float32)
true_B = tf.convert_to_tensor([[0.1, -2.], [-2., 0.1]], dtype=tf.float32)
d = [0., 0.5]

In [ ]:
class Lambda(tf.keras.Model):

    def call(self, y, t, d):

        By_integ = true_eqn_integrate(true_B, y, t - d[1], t - d[0])

        return tf.cast(tf.einsum('ab, cb -> ca', tf.cast(tf.transpose(true_A), tf.float64), tf.cast(y(t), tf.float64)) - 0.5*tf.einsum('ab, cb -> ca', tf.cast(tf.transpose(true_A), tf.float64), By_integ.integ), tf.float32)

In [ ]:
true_z = ddeinttf(Lambda(), true_z0, t, fargs=(d,))  # Solve for the true ODE solution

### Create validation set

In [ ]:
val_obj = nddde.create_validation_set_nddde(true_z0, t, args_nddde)

val_true_z = val_obj.data(Lambda(), true_z, t, d)

## Main

### Make objects and define learning-rate schedule

In [ ]:
end = time.time()
time_meter = nddde.RunningAverageMeter(0.97)

func_main = DDEFuncMain()
func_aux = DDEFuncAux()
func = nddde.DistDDEFunc(func_main, func_aux, args_nddde)
adj_func = nddde.nddde_adj_eqn(func, args_nddde)

get_batch = nddde.create_batch(true_z, true_z0, t, args_nddde)
loss_obj = custom_loss()
plot_obj = custom_plot(true_z, t, figsave_dir, args_nddde)
loss_history = nddde.history(args_nddde)

initial_learning_rate = args_nddde.learning_rate
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=args_nddde.niters,
    decay_rate=args_nddde.decay_rate,
    staircase=True)

### Quick test to see how the true DistDDE looks like

In [ ]:
if args_nddde.restart == 1: 
    func_main.load_weights(tf.train.latest_checkpoint(checkpoint_dir_main))
    func_aux.load_weights(tf.train.latest_checkpoint(checkpoint_dir_aux))
    process_true_z0 = nddde.process_DistDDE_IC(true_z0, func_aux, t_lowerlim = 0. - args_nddde.nn_d2, t_upperlim = 0. - args_nddde.nn_d1)
    pred_z = ddeinttf(func, process_true_z0, t, fargs=([args_nddde.nn_d1, args_nddde.nn_d2],))
    
    plot_obj.plot(pred_z, epoch = 0)

    loss_history.read()
    
    initial_learning_rate = 0.05
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate, decay_steps=args_nddde.niters, decay_rate=0.95, staircase=True)
    
else:
    plot_obj.plot(epoch = 0) 

### Training starts here

In [ ]:
optimizer_main = tf.keras.optimizers.RMSprop(learning_rate = lr_schedule)
optimizer_aux = tf.keras.optimizers.RMSprop(learning_rate = lr_schedule)

nDistDDE_train_obj = nddde.train_nDistDDE(func = func, adj_func = adj_func, d = [args_nddde.nn_d1, args_nddde.nn_d2], loss_obj = loss_obj, batch_obj = get_batch,
                            checkpoint_dir_aux = checkpoint_prefix_aux, optimizer_main = optimizer_main, optimizer_aux = optimizer_aux, args = args_nddde, plot_obj = plot_obj, time_meter = time_meter, checkpoint_dir_main = checkpoint_prefix_main,
                            validation_obj = val_obj, loss_history_obj = loss_history)

nDistDDE_train_obj.train(true_z, true_z0, t, val_true_z)